# 1. Recol·lecció i tractament de dades

 * 1.1 Executem la llibreria LabelMe, que manualment ens permetrà obtenir unes coordenades que marquin la àrea del objecte que volem detectar. Aquesta llibreria ens retornarà un arxiu .json amb els valors numèrics dels punts marcats. Amb aquests valors, la xarxa neuronal serà capaç d'entrenar-se. Haurem de crear una carpeta que anomenarem "labels" on hi aniran aquestes "etiquetes" en format .json

In [ ]:
!labelme #execució de la llibreria labelme

# 2. Creació de la base de dades

In [3]:
#importació de llibreries necessàries
import tensorflow as tf
import json
import numpy as np
from matplotlib import pyplot as plt
import os

In [ ]:
#limitació de la memòria GPU 
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
imatges = tf.data.Dataset.list_files('data\\images\\*.png') # introduïm les imatges a la base de dades

In [ ]:
def carregar_imatge(imatge): #funció per a carregar imatges
    byte_img = tf.io.read_file(imatge)
    img = tf.io.decode_png(byte_img)
    return img

In [ ]:
imatges = imatges.map(carregar_imatge) # executem la funció per carregar les imatges en la base de dades 

In [ ]:
imatges.as_numpy_iterator().next() 

In [ ]:
def quantImatgesDividir(dir_carpeta):
    num_elements = len(os.listdir(dir_carpeta))
    n_entrenar = round(num_elements*0.7) # 70% de les imatges per entrenar
    n_provar = round(num_elements*0.15) # 15% de les imatges per provar
    n_validar = num_elements - (n_entrenar + n_provar) # 15% de les imatges per validar
    print(f"{n_entrenar} --> 'train' \n {n_provar} --> 'test' \n {n_validar} --> 'val'")


In [ ]:
quantImatgesDividir('T:\REP\deteccio_prova\data\images') # s'han de dividir manualment les imatges (intentant que sigui aleatori)

In [ ]:
# mou les els labels a les seves respectives carpetes
for carpeta in ['train','test','val']:
    for arxiu in os.listdir(os.path.join('data', carpeta, 'images')): #per cada arxiu en cada carpeta
        nom_arxiu = arxiu.split('.')[0]+'.json' #extreu el nom de l'arxiu
        dir_existent = os.path.join('data','labels', nom_arxiu)#crea possible direcció
        if os.path.exists(dir_existent): #si la possible direcció existeix, desplaça l'arxiu .json corresponent
            nova_dir = os.path.join('data',carpeta,'labels',nom_arxiu)
            os.replace(dir_existent, nova_dir)#canviem

### 2.1 Augmentem la quantitat de dades amb la llibreria Albumentation

In [4]:
import albumentations as alb
import cv2

In [ ]:
augmentor = alb.Compose([alb.RandomCrop(width=1024, height=1024),
                         alb.HorizontalFlip(p=0.5), 
                         alb.RandomBrightnessContrast(p=0.2),
                         alb.RandomGamma(p=0.2), 
                         alb.RGBShift(p=0.2), 
                         alb.VerticalFlip(p=0.5)], 
                       bbox_params=alb.BboxParams(format='albumentations', #format de coords que volem (buscar docs)
                                                  label_fields=['class_labels']))

In [ ]:
for partition in ['train','test','val']: 
    for image in os.listdir(os.path.join('data', partition, 'images')):
        img = cv2.imread(os.path.join('data', partition, 'images', image))

        coords = [0,0,0.00001,0.00001]
        label_path = os.path.join('data', partition, 'labels', f'{image.split(".")[0]}.json')
        if os.path.exists(label_path):
            with open(label_path, 'r') as f:
                label = json.load(f)

            coords[0] = label['shapes'][0]['points'][0][0]
            coords[1] = label['shapes'][0]['points'][0][1]
            coords[2] = label['shapes'][0]['points'][1][0]
            coords[3] = label['shapes'][0]['points'][1][1]
            coords = list(np.divide(coords, [1024,1024,1024,1024]))
            # ^^^ carregar imatges i json ^^^

        try: 
            for x in range(60):# quantitat d'imatges que surten d'una imatge base
                augmented = augmentor(image=img, bboxes=[coords], class_labels=['Maduixa'])
                cv2.imwrite(os.path.join('aug_data', partition, 'images', f'{image.split(".")[0]}.{x}.jpg'),augmented['image'])
                
                annotation = {}
                annotation['image'] = image

                if os.path.exists(label_path):
                    if len(augmented['bboxes']) == 0: 
                        annotation['bbox'] = [0,0,0,0]
                        annotation['class'] = 0 
                    else: 
                        annotation['bbox'] = augmented['bboxes'][0]
                        annotation['class'] = 1
                else: 
                    annotation['bbox'] = [0,0,0,0]
                    annotation['class'] = 0 
                    '''
                     La variable “annotation” és un diccionari que conté informació sobre la imatge i 
                     la seva anotació. Si el fitxer d’anotació existeix, el codi afegeix la caixa delimitadora
                     i la classe corresponent a l’objecte detectat a l’estructura “annotation”. 
                     Si el fitxer d’anotació no existeix, el codi assigna una caixa delimitadora i 
                     una classe buida a l’estructura “annotation”.
                    '''
                with open(os.path.join('aug_data', partition, 'labels', f'{image.split(".")[0]}.{x}.json'), 'w') as f:
                    json.dump(annotation, f)

        except Exception as e:
            print(e)